In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres, convertir_fechaeventos
import psycopg2
import datetime

In [2]:
df_lesiones = pd.read_csv("data/lesiones_completo_normalizado.csv")

In [3]:
df_lesiones.rename(columns = {
    'JUGADOR': 'Nombre_Jugador',
    'href': 'Nombre_Completo_Jugador',
    'equipo': 'Club',
    'TIPO': 'Tipo_Lesion',
    'SUB TIPO': 'Sub_Tipo_Lesion',
    'TIPO_ID': 'Tipo_Lesion_ID',
    'SUB_TIPO_ID': 'Sub_Tipo_Lesion_ID',
    'DATA': 'Fecha_incio_lesion',
    'FIM': 'Fecha_fin_lesion'
}, inplace = True)

In [4]:
df_lesiones = convertir_fechaeventos(df_lesiones)

In [5]:
df_lesiones = df_lesiones.dropna(subset=["playerID"])
df_lesiones["playerID"] = df_lesiones["playerID"].astype(int)
df_lesiones["clubID"] = df_lesiones["clubID"].astype(int)

In [6]:
df_lesiones.dtypes

Nombre_Jugador             object
Tipo_Lesion                object
Sub_Tipo_Lesion            object
Fecha_incio_lesion         object
Fecha_fin_lesion           object
Nombre_Completo_Jugador    object
Club                       object
playerID                    int64
clubID                      int64
Tipo_Lesion_ID              int64
Sub_Tipo_Lesion_ID          int64
dtype: object

In [7]:
df_lesiones = df_lesiones[df_lesiones["Tipo_Lesion"] == "Injury"]

In [8]:
df_lesiones.drop(columns=(["Tipo_Lesion_ID", "Tipo_Lesion"]), inplace=True)

In [12]:
df_lesiones.rename(columns = {
    'Sub_Tipo_Lesion': 'Tipo_Lesion',
    'Sub_Tipo_Lesion_ID': 'Tipo_Lesion_ID',
}, inplace = True)

In [13]:
tabla_indice_lesiones = df_lesiones[["Tipo_Lesion", "Tipo_Lesion_ID"]].drop_duplicates()

In [17]:
df_lesiones["ID_Lesion"] = df_lesiones["Nombre_Jugador"].index

In [19]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_lesiones,"lesiones", cur, conn)
insertar_datos_automatico(tabla_indice_lesiones,"Lesiones_indice", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [20]:
df_lesiones.to_csv("data/Tabla_Lesiones.csv", index=False)
tabla_indice_lesiones.to_csv("data/Tabla_Lesiones_indice.csv", index=False)

In [33]:
df_lesiones = pd.read_csv("data/Tabla_Lesiones.csv")

In [34]:
tabla_indice_lesiones = pd.read_csv("data/Tabla_Lesiones_indice.csv")

In [35]:
mapa_lesiones_grupo = {
    # 1. Afecciones Abdominales y Gastrointestinales
    29: "Afecciones Abdominales y Gastrointestinales",  # Stomach Problems
    62: "Afecciones Abdominales y Gastrointestinales",  # Abdominal Problems

    # 2. Afecciones del Tendón de Aquiles
    33: "Afecciones del Tendón de Aquiles",  # Achilles tendon rupture
    44: "Afecciones del Tendón de Aquiles",  # Achilles Heel Problems
    45: "Afecciones del Tendón de Aquiles",  # Achilles tendon problems
    61: "Afecciones del Tendón de Aquiles",  # Achilles Tendon Irritation

    # 3. Lesiones de Cadera
    54: "Lesiones de Cadera",  # Hip Injury
    65: "Lesiones de Cadera",  # Hip Bruise

    # 4. Lesiones de Cabeza, Cara y Cuello
    13: "Lesiones de Cabeza, Cara y Cuello",  # Neck Injury
    57: "Lesiones de Cabeza, Cara y Cuello",  # Cheekbone Fracture
    68: "Lesiones de Cabeza, Cara y Cuello",  # Facial Fracture

    # 5. Lesiones de Espalda
    55: "Lesiones de Espalda",  # Back Injury

    # 6. Lesiones de Hombro y Clavícula
    46: "Lesiones de Hombro y Clavícula",  # Broken Collarbone
    70: "Lesiones de Hombro y Clavícula",  # Shoulder Injury

    # 7. Lesiones de Ingle, Aductores y Abductores
    3:  "Lesiones de Ingle, Aductores y Abductores",  # Adductor Pain
    6:  "Lesiones de Ingle, Aductores y Abductores",  # Tear in the abductor muscle
    15: "Lesiones de Ingle, Aductores y Abductores",  # Groin Injury
    38: "Lesiones de Ingle, Aductores y Abductores",  # Adductor Injury
    53: "Lesiones de Ingle, Aductores y Abductores",  # Groin Problems

    # 8. Lesiones de Ligamentos (Generales/No Articulares)
    31: "Lesiones de Ligamentos (Generales/No Articulares)",  # Ligament Stretching
    32: "Lesiones de Ligamentos (Generales/No Articulares)",  # Outer Ligament Problems

    # 9. Lesiones de Mano y Dedos
    47: "Lesiones de Mano y Dedos",  # Hand Injury
    50: "Lesiones de Mano y Dedos",  # Thumb Injury

    # 10. Lesiones de Muslo e Isquiotibiales
    28: "Lesiones de Muslo e Isquiotibiales",  # Hamstring Injury
    30: "Lesiones de Muslo e Isquiotibiales",  # Hamstring Strain
    34: "Lesiones de Muslo e Isquiotibiales",  # Thigh Problems
    67: "Lesiones de Muslo e Isquiotibiales",  # Thigh Muscle Strain

    # 11. Lesiones de Pierna Inferior (Pantorrilla y Huesos)
    2:  "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Calf Injury
    11: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Calf Problems
    35: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Broken Fibula
    41: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Broken Leg
    42: "Lesiones de Pierna Inferior (Pantorrilla y Huesos)",  # Leg Injury

    # 12. Lesiones de Rodilla (Complejo y Contusiones)
    10: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Problems
    18: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Bruise
    36: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Medial Ligament Tear
    37: "Lesiones de Rodilla (Complejo y Contusiones)",  # Knee Injury
    48: "Lesiones de Rodilla (Complejo y Contusiones)",  # Cruciate Ligament Injury
    49: "Lesiones de Rodilla (Complejo y Contusiones)",  # Meniscus Tear
    52: "Lesiones de Rodilla (Complejo y Contusiones)",  # Cruciate Ligament Tear
    58: "Lesiones de Rodilla (Complejo y Contusiones)",  # Twisted knee
    59: "Lesiones de Rodilla (Complejo y Contusiones)",  # Meniscus Injury

    # 13. Lesiones de Tobillo, Pie y Dedos
    4:  "Lesiones de Tobillo, Pie y Dedos",  # Ankle Problems
    12: "Lesiones de Tobillo, Pie y Dedos",  # Foot Injury
    22: "Lesiones de Tobillo, Pie y Dedos",  # Ankle Injury
    26: "Lesiones de Tobillo, Pie y Dedos",  # Toe Injury
    27: "Lesiones de Tobillo, Pie y Dedos",  # Ankle Sprain
    40: "Lesiones de Tobillo, Pie y Dedos",  # Broken Toe

    # 14. Lesiones Torácicas (Costillas, Pulmón)
    25: "Lesiones Torácicas (Costillas, Pulmón)",  # Lung Contusion
    43: "Lesiones Torácicas (Costillas, Pulmón)",  # Bruised Ribs

    # 15. Otros y Afecciones Generales
    5:  "Otros y Afecciones Generales",  # Rest
    7:  "Otros y Afecciones Generales",  # Muscle Strain
    8:  "Otros y Afecciones Generales",  # Muscle Injury
    14: "Otros y Afecciones Generales",  # Torn Muscle Bundle
    16: "Otros y Afecciones Generales",  # Knock
    19: "Otros y Afecciones Generales",  # Muscular problems
    20: "Otros y Afecciones Generales",  # Torn Muscle Fiber
    21: "Otros y Afecciones Generales",  # Minor Knock
    39: "Otros y Afecciones Generales",  # Fitness
    51: "Otros y Afecciones Generales",  # Infection
    60: "Otros y Afecciones Generales",  # Flu
    63: "Otros y Afecciones Generales",  # Virus
    64: "Otros y Afecciones Generales",  # Fever
    69: "Otros y Afecciones Generales"   # Unknown Injury
}

In [36]:
tabla_indice_lesiones["Tipo_Lesion"] = tabla_indice_lesiones["Tipo_Lesion_ID"].map(mapa_lesiones_grupo)

In [37]:
mapa_nombre_grupo_a_nuevo_id = {
    "Afecciones Abdominales y Gastrointestinales": 1,
    "Afecciones del Tendón de Aquiles": 2,
    "Lesiones de Cadera": 3,
    "Lesiones de Cabeza, Cara y Cuello": 4,
    "Lesiones de Espalda": 5,
    "Lesiones de Hombro y Clavícula": 6,
    "Lesiones de Ingle, Aductores y Abductores": 7,
    "Lesiones de Ligamentos (Generales/No Articulares)": 8,
    "Lesiones de Mano y Dedos": 9,
    "Lesiones de Muslo e Isquiotibiales": 10,
    "Lesiones de Pierna Inferior (Pantorrilla y Huesos)": 11,
    "Lesiones de Rodilla (Complejo y Contusiones)": 12,
    "Lesiones de Tobillo, Pie y Dedos": 13,
    "Lesiones Torácicas (Costillas, Pulmón)": 14,
    "Otros y Afecciones Generales": 15
}

In [38]:
tabla_indice_lesiones["Tipo_Lesion_ID"] = tabla_indice_lesiones["Tipo_Lesion"].map(mapa_nombre_grupo_a_nuevo_id)

In [39]:
tabla_indice_lesiones = tabla_indice_lesiones.groupby("Tipo_Lesion_ID").count().reset_index()

In [40]:
mapa_nuevo_id = {
   1: "Abdomen izquierdo",
    2: "Pantorrilla derecha",
    3: "Glúteo izquierdo",
    4: "Cabeza delantera",
    5: "Erector de la columna izquierdo",
    6: "Deltoide izquierdo delantero",
    7: "Aductor largo izquierdo",
    8: "Recto femoral izquierdo",
    9: "Flexor Digitorum Izquierdo",
    10: "Bíceps femoral izquierdo",
    11: "Pantorrilla izquierda",
    12: "Rodilla izquierda delantera",
    13: "Tibial anterior izquierdo",
    14: "Erector de la columna derecho",
    15: "Cabeza trasera"
}

In [41]:
tabla_indice_lesiones["Tipo_Lesion"] = tabla_indice_lesiones["Tipo_Lesion_ID"].map(mapa_nuevo_id)

In [43]:
tabla_indice_lesiones["Tipo_Lesion"].unique()

array(['Abdomen izquierdo', 'Pantorrilla derecha', 'Glúteo izquierdo',
       'Cabeza delantera', 'Erector de la columna izquierdo',
       'Deltoide izquierdo delantero', 'Aductor largo izquierdo',
       'Recto femoral izquierdo', 'Flexor Digitorum Izquierdo',
       'Bíceps femoral izquierdo', 'Pantorrilla izquierda',
       'Rodilla izquierda delantera', 'Tibial anterior izquierdo',
       'Erector de la columna derecho', 'Cabeza trasera'], dtype=object)

In [34]:
df_lesiones

,Nombre_Jugador,Tipo_Lesion,Fecha_incio_lesion,Fecha_fin_lesion,Nombre_Completo_Jugador,Club,playerID,clubID,Tipo_Lesion_ID,ID_Lesion
0,Claudio Bravo,Calf Injury,2016-05-02,2016-05-19,Claudio Andrés Bravo Muñoz,Barcelona,4691,217,2,2
1,Aleix Vidal,Adductor Pain,2016-04-04,2016-04-14,Aleix Vidal Parreu,Barcelona,6400,217,3,5
2,Jordi Alba,Ankle Problems,2016-03-27,2016-04-02,Jordi Alba Ramos,Barcelona,5211,217,4,6
3,Ivan Rakitic,Calf Injury,2016-03-20,2016-04-02,Ivan Rakitić,Barcelona,5470,217,2,8
4,Sergio Busquets,Rest,2016-03-10,2016-03-14,Sergio Busquets i Burgos,Barcelona,5203,217,5,9
...,...,...,...,...,...,...,...,...,...,...
191,Emiliano Velázquez,Muscle Injury,2016-03-18,2016-05-08,Emiliano Daniel Velázquez Maldonado,Getafe,17054,216,8,363
192,Pablo Sarabia,Shoulder Injury,2016-01-29,2016-02-11,Pablo Sarabia García,Getafe,6720,216,70,368
193,Carlos Vigaray,Calf Injury,2015-11-03,2015-11-17,Carlos Martín Vigaray,Getafe,6856,216,2,370
194,Álvaro Medrán,Broken Leg,2015-10-09,2016-02-08,Álvaro Medrán Just,Getafe,6736,216,41,371


In [44]:
df_lesiones["Tipo_Lesion"] = df_lesiones["Tipo_Lesion_ID"].map(mapa_lesiones_grupo)
df_lesiones["Tipo_Lesion_ID"] = df_lesiones["Tipo_Lesion"].map(mapa_nombre_grupo_a_nuevo_id)
df_lesiones["Tipo_Lesion"] = df_lesiones["Tipo_Lesion_ID"].map(mapa_nuevo_id)

In [45]:
df_lesiones["Tipo_Lesion"].unique()

array(['Pantorrilla izquierda', 'Aductor largo izquierdo',
       'Tibial anterior izquierdo', 'Cabeza trasera',
       'Rodilla izquierda delantera', 'Cabeza delantera',
       'Erector de la columna derecho', 'Bíceps femoral izquierdo',
       'Abdomen izquierdo', 'Recto femoral izquierdo',
       'Pantorrilla derecha', 'Deltoide izquierdo delantero',
       'Flexor Digitorum Izquierdo', 'Glúteo izquierdo',
       'Erector de la columna izquierdo'], dtype=object)

In [46]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_lesiones,"lesiones", cur, conn)
insertar_datos_automatico(tabla_indice_lesiones,"Lesiones_indice", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [47]:
df_lesiones.to_csv("data/Tabla_Lesiones.csv", index=False)
tabla_indice_lesiones.to_csv("data/Tabla_Lesiones_indice.csv", index=False)